In [1]:
# get list of urls for all regions
# for each url
# get list of rows
# for each row process row
# turn each processsed list of rows into dataframe
# concatanate dataframes together
# perform post processing cleanup

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import os

In [3]:
url = "https://www.ss.com/lv/real-estate/farms-estates/"

In [4]:
def getUrlList(url, prefix='https://www.ss.com', postfix='sell/', tag='a', class_='a_category'):
    """
    Returns a list of href links from a url
    """
    req = requests.get(url)
    if req.status_code != 200:
        print(f'Unexpected status code {req.status_code}. Stopping parse')
        return [] #return early and often principle
    soup = BeautifulSoup(req.text, 'lxml') # could skip soup variable as well but keeping for readability
    return [ prefix + el['href'] + postfix for el in soup.find_all(tag, class_) ]
    # What else could we pass as argument? How could our return fail?

In [15]:
def getRowData(row, baseurl="https://www.ss.com"):
    return [el.text for el in row.find_all('td')[2:]] + [baseurl + row.find('a')['href']]

In [25]:
def getDFfromUrl(url, region = None):
    """
    # get list of rows
    # for each row process row
    # turn each processsed list of rows into dataframe
    """
    print(f'Going to gather data from URL:{url}')
    req = requests.get(url)
    if req.status_code != 200:
        print(f'Unexpected status code {req.status_code}. Stopping parse')
        return [] #return early and often principle
    soup = BeautifulSoup(req.text, 'lxml') # could skip soup variable as well but keeping for readability
        
    headline = soup.find('tr', id = "head_line")
    cindex = [el.text for el in headline.find_all('td')]
    cindex[0] = cindex[0].split()[0]
    cindex += ['URL'] #TODO add argument for this
    cindex += ['Region']
    
    # TODO move it somewhere else
    if len([el for el in soup.find_all('option') if 'Pārdod' in el.text]) == 0:
        print("Oops nothing for sale")
        return pd.DataFrame({}, columns=cindex)
    
    rows = soup.find_all('tr',id = re.compile(r'tr_[0-9]+'))
    rowsdata = [getRowData(el) for el in rows] 
    # finally we add the region if we did not have one
    if region == None:
        region = url.split("/")[-3]
    # we add region information at the end of our list
    rowsdata = [el + [region] for el in rowsdata]
    return pd.DataFrame(rowsdata, columns=cindex)

In [6]:
# with this recipe we can append a big list of dataframes into one
def getDFfromUrlList(urlist, delay=0.5):
    dflist = []
    for ur in urlist:
        dflist.append(getDFfromUrl(ur))
        time.sleep(delay)
    return pd.concat(dflist)

In [8]:
def cleanDF(df):
    # for each price element, i split by empty space, get begging, get rid of comma, then cast to int
    df[['Stāvi', 'm2']] = df[['Stāvi', 'm2']].apply(pd.to_numeric, errors='coerce', axis=1)
    df['Price'] = df['Cena'].apply(lambda el: int(el.split(" ")[0].replace(',','')))
    # bonus, create currency column
    df['Currency'] = df['Cena'].apply(lambda el: el.split(" ")[-1])
    return df

In [22]:
os.path.join("C:\\mansprojekts\\", "mansfails.txt")

'C:\\mansprojekts\\mansfails.txt'

In [23]:
def saveDF(df, path=None, name="Farmhouses", my_sheet_name="Sheet_1"):
    # https://stackoverflow.com/questions/10607688/how-to-create-a-file-name-with-the-current-date-time-in-python
    timestr = time.strftime("%Y%m%d-%H%M%S")
    # full way of writing to excel, could just do the short df.to_excel(filename)
    fname = f'{name}_{timestr}.xlsx'
    if path:
        fullpath = os.path.join(path, fname)
    else:
        fullpath = fname
        
    with pd.ExcelWriter(fullpath) as writer:
        df.to_excel(writer, sheet_name=my_sheet_name)

In [31]:
farmurls = getUrlList(url)
#df = getDFfromUrlList(farmurls)
df = getDFfromUrlList(farmurls[:-1]) # we do not reallyneed the Cits region
df = cleanDF(df)
saveDF(df)

Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/riga-region/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/aizkraukle-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/aluksne-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/balvi-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/bauska-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/cesis-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/daugavpils-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/dobele-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/gulbene-and-reg/sell/
Going to gather data from URL:https://www.ss.com/lv/real-estate/farms-estates/jekabpils-and-reg/sell/
Go

In [33]:
df.shape

(275, 10)

In [28]:
os.getcwd()

'C:\\Users\\val-p1\\Github\\RCS_Data_Analysis_Python_11_2019\\WebScraping'

In [30]:
saveDF(df, "..") # this will save one map above current map 